In [16]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

#import packages
import pandas as pd
from pycmtensor.dataset import Dataset
from pycmtensor.expressions import Beta, RandomDraws
from pycmtensor.models import MNL

lpmc = pd.read_csv("../data/lpmc.dat", sep="\t")
lpmc = lpmc[lpmc["travel_year"]==2015]
ds = Dataset(df=lpmc, choice="travel_mode")
ds.split(0.8)

# Beta parameters
asc_walk = Beta("asc_walk", 0.0, None, None, 1)
asc_cycle = Beta("asc_cycle", 0.0, None, None, 0)
asc_pt = Beta("asc_pt", 0.0, None, None, 0)
asc_drive = Beta("asc_drive", 0.0, None, None, 0)
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
s_time = Beta("s_time", 0.5, None, None, 0)
b_purpose = Beta("b_purpose", 0.0, None, None, 0)
b_licence = Beta("b_licence", 0.0, None, None, 0)
b_car_own = Beta("b_car_own", 0.0, None, None, 0)

rnd_time = RandomDraws('rnd_time', 'lognormal', 100)
U_walk = asc_walk + b_time * ds["dur_walking"] # + rnd_time(s_time) * db["dur_walking"]
U_cycle = asc_cycle + b_time * ds["dur_cycling"] 
U_pt = asc_pt + b_time * (ds["dur_pt_rail"] + ds["dur_pt_bus"] + ds["dur_pt_int"]) + b_cost * ds["cost_transit"]
U_drive = asc_drive + b_time * ds["dur_driving"] + b_cost * (ds["cost_driving_fuel"] + ds["cost_driving_ccharge"]) + b_licence * ds["driving_license"]

U = [U_walk, U_cycle, U_pt, U_drive]

15:49:49 [INFO] n_train_samples:3986 n_valid_samples:997


In [71]:
import aesara.tensor as aet
import aesara
import numpy as np

inputs = [ds['age'], ds['female'], ds['car_ownership']]

# z variable
z = aet.stack(*inputs)
# weights
W = aesara.shared(0.1* np.random.normal(size=(3, 10)))
W2 = aesara.shared(0.1* np.random.normal(size=(10, 1)))

# bias
bias = aesara.shared(np.zeros((10, 1)))
bias2 = aesara.shared(np.zeros((1, 1)))

h = aet.softplus(W.T @ z) + bias

b_licence_TN = aet.softplus(W2.T @ h) + bias2
license_out = b_licence_TN * ds["driving_license"]

U_walk = asc_walk + b_time * ds["dur_walking"] # + rnd_time(s_time) * db["dur_walking"]
U_cycle = asc_cycle + b_time * ds["dur_cycling"] 
U_pt = asc_pt + b_time * (ds["dur_pt_rail"] + ds["dur_pt_bus"] + ds["dur_pt_int"]) + b_cost * ds["cost_transit"]
U_drive = asc_drive + b_time * ds["dur_driving"] + b_cost * (ds["cost_driving_fuel"] + ds["cost_driving_ccharge"]) + b_licence * ds["driving_license"] + license_out

U = [U_walk, U_cycle, U_pt, U_drive]

from pycmtensor.functions import logit, log_likelihood
import pycmtensor
from pycmtensor.models.basic import extract_params

p_y_given_x = logit(U)
ll = log_likelihood(p_y_given_x, ds.y)
cost = - ll / ds.y.shape[0]

params = extract_params(cost, locals())
optimizer = pycmtensor.config.optimizer(params)
updates = optimizer.update(cost, params, 0.1)

params = [p() for p in params if (p.status!=1)]
grads = aet.grad(cost, license_out)

fn = aesara.function(
    inputs=ds.x+[ds.y], 
    outputs=grads
)

data = ds.train_dataset(ds.x+[ds.y], index=7, batch_size=32)
# print(np.stack(data))
fn(*data)

16:12:02 [WARNING] s_time not in any utility functions
16:12:02 [WARNING] b_purpose not in any utility functions
16:12:02 [WARNING] b_car_own not in any utility functions


array([[ 0.01647871,  0.0078125 , -0.01186443, -0.01236618,  0.01756461,
         0.0078125 , -0.0234375 ,  0.0078125 ,  0.01956647, -0.00968083,
         0.0078125 ,  0.02041445, -0.01535953, -0.0084862 ,  0.0078125 ,
        -0.013743  ,  0.01689609,  0.01573495, -0.0234375 ,  0.0078125 ,
         0.0078125 ,  0.01868121,  0.0078125 , -0.01521247, -0.01432622,
         0.0078125 ,  0.01871448, -0.01240953, -0.00743806,  0.0078125 ,
         0.0078125 ,  0.0078125 ]])

In [52]:
import aesara.tensor as aet
import aesara
import numpy as np

x = aet.join(0, ds["cost_driving_fuel"], ds["cost_driving_ccharge"])
print(x.ndim)  # (2,)

y = aet.as_tensor_variable([ds["cost_driving_fuel"], ds["cost_driving_ccharge"]])
print(y.ndim)  # (2, N)

# weights
W = aesara.shared(np.random.lognormal(size=(2, 6)))

W2 = aesara.shared(np.random.lognormal(size=(6, 1)))

# bias
bias = aesara.shared(np.zeros((6, 1)))
bias2 = aesara.shared(np.zeros((1, 1)))

h = aet.softplus(W.T @ y) + bias

h2 = aet.softplus(W2.T @ h) + bias2

# h2 = aet.flatten(h2)

out = h2 * ds["cost_driving_fuel"]

u = [out, out]
u = aet.stack(u)

fn = aesara.function([ds["cost_driving_fuel"], ds["cost_driving_ccharge"]], u)

data =ds.train_dataset([ds["cost_driving_fuel"], ds["cost_driving_ccharge"]], index=7, batch_size=10)

fn(*data)

1
2


array([[[ 68.57159147,   6.87060593,  18.08116322, 322.68926133,
           0.91667335,   3.23301396,  64.86097475,  18.08193963,
           3.85082289,   0.98586741]],

       [[ 68.57159147,   6.87060593,  18.08116322, 322.68926133,
           0.91667335,   3.23301396,  64.86097475,  18.08193963,
           3.85082289,   0.98586741]]])